In [50]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import nltk
import boto3
from botocore.exceptions import ClientError
import os
import json
import pymorphy2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

In [2]:
twitter_data = pd.read_csv("Corona_NLP_train.csv", encoding = "ISO-8859-1")
twitter_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


Для начала немного преподготовим наши данные, и объеденим Extremely Negative с Negative  
и Extremely Positive с Positive

In [3]:
twitter_data.loc[twitter_data.Sentiment == 'Extremely Negative', 'Sentiment'] = 'Negative'
twitter_data.loc[twitter_data.Sentiment == 'Extremely Positive', 'Sentiment'] = 'Positive'

In [4]:
twitter_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Negative


Теперь очистим данные от хэштегов, ссылок и прочего

In [5]:
def clean(data):
    out = []
    for i in range(0, len(data)):
        temp = data[i]
        temp = re.sub("@\S+", " ", temp)
        temp = re.sub("https*\S+", " ", temp)
        temp = re.sub("#\S+", " ", temp)
        temp = re.sub("\'\w+", '', temp)
        temp = re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
        temp = re.sub(r'\w*\d+\w*', '', temp)
        temp = re.sub('\s{2,}', " ", temp)
        list_of_temp = temp.split(" ")
        list_of_temp = list(map(lambda x: x.lower(), list_of_temp))
        temp = ' '.join(list_of_temp)
        out.append(temp)
    return out

In [6]:
out = clean(list(twitter_data['OriginalTweet']))

Создадим новый датасет, с которым и будем работать в будущем.  
Для определения тональности текста нам не понадобятся username, ScreenName, Location, TweetAt

In [8]:
clean_twitts = pd.DataFrame()

clean_twitts['OriginalTweet'] = out
clean_twitts['Sentiment'] = twitter_data['Sentiment']

clean_twitts.head()

,OriginalTweet,Sentiment
0,and and,Neutral
1,advice talk to your neighbours family to excha...,Positive
2,coronavirus australia woolworths to give elder...,Positive
3,my food stock is not the only one which is emp...,Positive
4,me ready to go at supermarket during the outbr...,Negative


In [9]:
# Посмотрим какие классы у нас вообще есть
list_of_labels = list(clean_twitts['Sentiment'].unique())
list_of_labels

['Neutral', 'Positive', 'Negative']

In [10]:
# функция разделяет твитты по классам и записывает их в файл, нужно мне для анализа в amazon comprehend
def get_twitts(twitter_data, label, filename):
    l = twitter_data[twitter_data.Sentiment == label]
    l = list(l['OriginalTweet'])
    
    f = open(f"{filename}.txt", "w")
    for twitt in l[0:int(len(l)/10)]:
        f.write(twitt + "\n")
    f.close()

In [12]:
# создадим файл для каждого класса
# в результате у нас есть три файла
# в каждом файле-классе только твитты относящиеся к этому классу
for label in list_of_labels:
    get_twitts(clean_twitts, label, label)

Далее а Amazon Comprehend я для каждого файла сделала анализ на Sentiment, по строчно.  
Так как у меня в каждом файле хранятся твитты (разделенные новой строкой), относящиеся только к классу с названием файла, метрику accuracy можно посчитать следующим образом

In [13]:
def get_accuracy(filename, clean=False):
    jsons = []
    with open(filename) as f:
        for line in f:
            jsons.append(json.loads(line))
    
    score = 0
    for j in jsons:
        # сравниваю название файла с предсказаной тональностью (должны совпадать)
        original_sentiment = j['File'].split(".")[0].lower()
        if clean:
            original_sentiment = original_sentiment.split("_")[0]
        pred_sentiment = j['Sentiment'].lower()
        if original_sentiment == pred_sentiment:
            score += 1
                        
    return score / len(jsons) * 100

Как мы видим с помощью амазон сервиса мы получили крайне низкий результат accuracy  
Лишь немного выше чем рандомный выбор с вероятностью угадать 1/3

In [14]:
accuracy = get_accuracy("sentiment_from_comprehend.txt")
print(f"Accuracy = {accuracy}%")

Accuracy = 39.499270782693245%


Теперь попробуем всё тоже самое, только удалим стоп-слова и преобразуем все слова в начальную форму.  
Для этого создадим ещё один датафрейм, потому что тот нам еще понадобится

In [16]:
# преобразуем в начальную форму и удаляем стоп-слова

list_of_twitts = list(twitter_data['OriginalTweet'])
out_list = []

morph = pymorphy2.MorphAnalyzer(lang='uk')

for twitt in list_of_twitts:
    # удаляем стоп-слова
    filtered_words = [word for word in twitt.split(" ") if word not in stopwords.words('english')]
    # преобразуем в начальную форму
    normal_form_words = []
    for word in filtered_words:
        normal_form_words.append(morph.parse(word)[0].normal_form)
    # объединяем
    out_list.append(' '.join(normal_form_words))

In [17]:
clean_and_normal = pd.DataFrame()

clean_and_normal['OriginalTweet'] = out_list
clean_and_normal['Sentiment'] = twitter_data['Sentiment']

clean_and_normal.head()

,OriginalTweet,Sentiment
0,@menyrbie @phil_gahan @chrisitv https://t.co/i...,Neutral
1,advice talk neighbours family exchange phone n...,Positive
2,coronavirus australia: woolworths give elderly...,Positive
3,"my food stock one empty...\r\r\n\r\r\nplease, ...",Positive
4,"me, ready go supermarket #covid19 outbreak.\r\...",Negative


In [19]:
# создадим такие же файлы как и раньше
for label in list_of_labels:
    get_twitts(clean_and_normal, label, label + '_cleanup')

Далее всё тоже самое с Amazon Comprehend  
Как можно наблюдать точность уменьшилась, что меня несколько удивило

In [20]:
accuracy = get_accuracy("sentiment_from_comprehend_clean_data.txt", True)
print(f"Accuracy = {accuracy}%")

Accuracy = 31.40495867768595%


Теперь попробуем написать что-то своё

In [34]:
# нам нужна функция которая посчитает метрику accuracy схожим способом что и раньше
def get_accuracy_from_list(real, predictions):
    score = 0
    for i in range(len(real)):
        if real[i] == predictions[i]:
            score += 1
    return score / len(real) * 100

Далее поделим наши данные на тестовую и обучающие выборки, сначала для данных, очищенных только от хэштегов, ссылок и прочего

In [21]:
X_train, X_test, y_train, y_test = train_test_split(clean_twitts['OriginalTweet'], clean_twitts['Sentiment'], test_size=0.2, random_state=42)

Преобразуем данные в вектор

In [22]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000)

In [24]:
train_data_features = vectorizer.fit_transform(list(X_train)).toarray()

In [51]:
scores = cross_val_score(LogisticRegression(), train_data_features, y_train, cv=5)

Как можно наблюдать даже простая логистическая регрессия справилась с нашей задачей куда лучаше

In [52]:
print("Accuracy: {:.2f}".format(np.mean(scores)))

Accuracy: 0.79


Всё тоже самое, только для нормализованных данных

In [53]:
X_train, X_test, y_train, y_test = train_test_split(clean_and_normal['OriginalTweet'], clean_and_normal['Sentiment'], test_size=0.2, random_state=42)
train_data_features = vectorizer.fit_transform(list(X_train)).toarray()
scores = cross_val_score(LogisticRegression(), train_data_features, y_train, cv=5)

In [54]:
print("Accuracy: {:.2f}".format(np.mean(scores)))

Accuracy: 0.79


С RandomForest

In [55]:
train_data_features = vectorizer.fit_transform(list(X_train)).toarray()
scores = cross_val_score(RandomForestClassifier(n_estimators = 200), train_data_features, y_train, cv=5)
print("Accuracy: {:.2f}".format(np.mean(scores)))

Accuracy: 0.74
